<a href="https://colab.research.google.com/github/ljtamang/codeBERT-QG/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install pacakages below:

In [2]:
!pip install torch
!pip install transformers
!pip install nltk
!pip install psutil
!pip install h5py 
!pip install typing-extensions 
!pip install wheel 
!pip install blue

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!git clone https://github.com/ljtamang/codeBERT-QG.git

Cloning into 'codeBERT-QG'...
remote: Enumerating objects: 240, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 240 (delta 93), reused 152 (delta 76), pack-reused 60
Receiving objects: 100% (240/240), 80.08 MiB | 17.44 MiB/s, done.
Resolving deltas: 100% (101/101), done.


Navigate inside the recently downloaded codeBERT-QG project folder

In [4]:
%cd codeBERT-QG
#%cd codeBERT-QG/code

/content/codeBERT-QG


In [ ]:
#when GPU 1 is available
!bash java_script.sh 0 cbQG


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
epoch 0 loss 6.9959:  17% 382/2282 [10:35<52:32,  1.66s/it]

In [ ]:
# Run the Test
#!bash java_script_test.sh 0 cbQG